## 1. Master notebook for output and result computations

In [1]:
import os
import time
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt; plt.style.use('dark_background')
from itertools import product
import subprocess
import tables
import yaml

In [2]:
ROOT = 'C:/Users/49176/Uni/Masterthesis/' 

# Path to local Anaconda prompt for subprocesses. Might require tweaking if another IDE is used.
# Main functionality that needs to be provided is starting a python script in a subprocess in
# a designated envrionment with arguments.
PYTHON_DIST = r'C:\Users\49176\anaconda3\Scripts\activate.bat'

In [3]:
with open("countrycode.yaml", 'r') as stream:
    countrycode = yaml.safe_load(stream)  
    
countries = countrycode.keys()

In [4]:
countries = list(countries)
scenarios = ['base']
urban_distances = ['1000','800','600', '400']
slopes = ['30', '15']
forest_shares = ['10','5', '0']
lpa_shares = ['10','5','0']
turbines = [('Vestas_V90_3MW', 'Vestas_V112_3MW'),('W2E_151_4800kW', 'W2E_165_5200kW')]

In [5]:
# Load functions to manage results and caluclations

def concat_turbine_tuple(turbine):
    
    if type(turbine) == tuple: 
        turbine_string = f'{turbine[0]}-{turbine[1]}'
    else:
        turbine_string = turbine

    return turbine_string

def access_hd5_keys(path):
    
    store = pd.HDFStore(path)
    keys = store.keys()
    store.close()
    
    return keys

class results_manager():

    def __init__(self, ROOT, countries, scenarios, urban_distances, slopes, forest_shares, lpa_shares, turbines): 
        
        self.ROOT = ROOT
        
        self.countries = countries
        self.scenarios = scenarios
        self.urban_distances = urban_distances
        self.slopes = slopes
        self.forest_shares = forest_shares
        self.lpa_shares = lpa_shares
        self.turbines = turbines
        self.combinations = len(countries)  * len(scenarios) * len(urban_distances) * len(slopes) * len(forest_shares) * len(lpa_shares) * len(turbines)
        
        
        # Check if preliminaries files exist
        
        # No reason to compute 0 forest/lpa share, but important in final result perception
        forest_shares_adj = [i for i in forest_shares if i != '0']
        lpa_shares_adj = [i for i in lpa_shares if i != '0']
       
        pre = {}       
        pre['country_shapes'] = {country : os.path.isfile(ROOT + f'data/country_shapes/{country}.gpkg') for country in countries}
        pre['landuse_pa'] = {country : os.path.isfile(ROOT + f'data/country_pa/{country}.gpkg') for country in countries}
        pre['landuse_slope'] = {f'{country}-{slope}' : os.path.isfile(ROOT + f'data/country_slope/{country}-{slope}.tif') for country,slope in product(countries, slopes)}
        pre['landuse_forest'] = {f'{country}-{scenario}-{urban_distance}-{slope}-{forest_share}' : os.path.isfile(ROOT + f'data/country_share_forest/{country}-{scenario}-{urban_distance}-{slope}-{forest_share}.tif') for country, scenario, urban_distance,slope,forest_share in product(countries, scenarios, urban_distances, slopes, forest_shares_adj)}
        pre['landuse_lpa'] = {f'{country}-{scenario}-{urban_distance}-{slope}-{lpa_share}' : os.path.isfile(ROOT + f'data/country_share_lpa/{country}-{scenario}-{urban_distance}-{slope}-{lpa_share}.tif') for country, scenario, urban_distance,slope,lpa_share in product(countries, scenarios, urban_distances, slopes, lpa_shares_adj)}
        pre['landuse_availability'] = {f'{country}-{scenario}-{urban_distance}-{slope}-{forest_share}-{lpa_share}' : os.path.isfile(ROOT + f'data/country_landuse_results/{country}-{scenario}-{urban_distance}-{slope}-{forest_share}-{lpa_share}.tif') for country,scenario,urban_distance,slope,forest_share,lpa_share in product(countries,scenarios,urban_distances,slopes,forest_shares,lpa_shares)}
        pre['era5_azimuth'] = {country : os.path.isfile(ROOT + f'data/country_azimuth/{country}-burned-azimuth.tif') for country in countries}
        pre['era5_turbine'] = {f'{country}-{turbine[0]}-{turbine[1]}' : os.path.isfile(ROOT + f'data/country_turbine_decision/{country}-{turbine[0]}-{turbine[1]}-burned-decision.tif') for country,turbine in product(countries, [turbine_tuples for turbine_tuples in turbines if isinstance(turbine_tuples, tuple)])}
        pre['turbine_placement'] = {f'{country}-{scenario}-{urban_distance}-{slope}-{forest_share}-{lpa_share}-{concat_turbine_tuple(turbine)}' : os.path.isfile(ROOT + f'data/country_turbine_placements/{country}-{scenario}-{urban_distance}-{slope}-{forest_share}-{lpa_share}-{concat_turbine_tuple(turbine)}.gpkg') for country,scenario,urban_distance,slope,forest_share,lpa_share,turbine in product(countries,scenarios,urban_distances,slopes,forest_shares,lpa_shares,turbines)}        
        
        self.pre = pre
        self.pre_true = {k:[k for k,v in self.pre[k].items() if v == True] for k in self.pre.keys()}
        self.pre_false = {k:[k for k,v in self.pre[k].items() if v == False] for k in self.pre.keys()}
        
        
        # Check if results exist      
        self.res_exist =  {country:[v.replace('/','') for v in access_hd5_keys(f'{ROOT}/data/country_generation_potential/{country}.h5')] for country in countries}
        self.res = {country:{f'{scenario}-{urban_distance}-{slope}-{forest_share}-{lpa_share}-{concat_turbine_tuple(turbine)}': f'{scenario}-{urban_distance}-{slope}-{forest_share}-{lpa_share}-{concat_turbine_tuple(turbine)}' in self.res_exist[country] for scenario,urban_distance,slope,forest_share,lpa_share,turbine in product(scenarios,urban_distances,slopes,forest_shares,lpa_shares,turbines)} for country in countries}
        self.res_true = {k:[k for k,v in self.res[k].items() if v == True] for k in self.res.keys()}
        self.res_false = {k:[k for k,v in self.res[k].items() if v == False] for k in self.res.keys()}
        
    def compute_prelims(self, arglist):
    
        activate_env= ['call', PYTHON_DIST, 'activate', 'atlite', '&&', 'python'] 
        
        combs = [product([k],[l.split('-') for l in v]) for k,v in arglist.items()]
        combs_list = [list(item) for sublist in combs for item in sublist]
        
        for module, args in combs_list:
            
            print('Running Module:', module, 'with args:', args)
            program_starts = time.time()
        
            run_script = [fr'{ROOT}windgenpotential/{module}.py', ROOT,  *args]

            p = subprocess.Popen(activate_env+run_script, 
                                 stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
                      
            out, err = p.communicate() 
            rc = p.returncode
                  
            if rc != 0:
                print('error code:', rc, ',', out, ',', err)
                break
            
            now = time.time()
            print("Computation time: {0} minutes \n".format(round((now - program_starts)/60),2))
                
    def compute_gen_potential(self, arglist):
        
        activate_env= ['call', PYTHON_DIST, 'activate', 'atlite', '&&', 'python']
        
        combs = [product([k],[l.split('-') for l in v]) for k,v in arglist.items()]
        combs_list = [list(item) for sublist in combs for item in sublist]
        
        for country, args in combs_list:
            
            print('Computing results for:', country, 'with args:', args)
            
            program_starts = time.time()
        
            run_script = [fr'{ROOT}windgenpotential/potential_calculation.py', ROOT, country, *args]

            p = subprocess.Popen(activate_env+run_script, 
                                 stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
                      
            out, err = p.communicate() 
            rc = p.returncode
                  
            if rc == 0:
                print('successfull')
            else:
                print('error code:', rc, ',', out.decode('UTF-8'), ',', err.decode('UTF-8'))
                break
                
            now = time.time()
            
            print("Computation time: {0} minutes \n".format(round((now - program_starts)/60),2))
        
        

In [6]:
# Initialize result manager
go = results_manager(ROOT, countries, scenarios, urban_distances, slopes, forest_shares, lpa_shares, turbines)

In [7]:
# Total number of scenarios to be calculated
go.combinations

5184

In [8]:
# Dictionary of missing preliminary results
d = go.pre_false
d

{'country_shapes': [],
 'landuse_pa': [],
 'landuse_slope': [],
 'landuse_forest': [],
 'landuse_lpa': [],
 'landuse_availability': [],
 'era5_azimuth': [],
 'era5_turbine': [],
 'turbine_placement': []}

In [34]:
# Compute missing preliminary results
%%time
go.compute_prelims(d)

CPU times: total: 0 ns
Wall time: 498 µs


In [9]:
# Dictionary of missing final results
d = go.res_false
d

{'Albania': [],
 'Austria': [],
 'Belgium': [],
 'Bosnia and Herzegovina': [],
 'Bulgaria': [],
 'Croatia': [],
 'Cyprus': [],
 'Czech Republic': [],
 'Denmark': [],
 'Estonia': [],
 'Finland': [],
 'France': [],
 'Germany': [],
 'Greece': [],
 'Hungary': [],
 'Iceland': [],
 'Ireland': [],
 'Italy': [],
 'Latvia': [],
 'Lithuania': [],
 'Luxembourg': [],
 'Malta': [],
 'Montenegro': [],
 'Netherlands': [],
 'Norway': [],
 'Poland': [],
 'Portugal': [],
 'North Macedonia': [],
 'Romania': [],
 'Serbia': [],
 'Slovakia': [],
 'Slovenia': [],
 'Spain': [],
 'Sweden': [],
 'Switzerland': [],
 'United Kingdom': []}

In [10]:
# Compute missing final results
go.compute_gen_potential(go.res_false)